In [12]:
import pandas as pd
import mysql.connector
from mysql.connector import Error
from urllib.parse import quote_plus
from sqlalchemy import create_engine
from geopy.geocoders import Nominatim
import time

In [13]:
# creation of connection and engine
def connection(host_name, user_name, user_password, dbname):
    connection = None
    user_password = quote_plus(user_password)
    try:
        connection = mysql.connector.connect(host=host_name, user=user_name, passwd = user_password, database  = dbname)
        print("Connection successful!")
    except Error as e:
        print(f"The error {e} has occured")
    return connection

def engine(host_name, user_name, user_password, dbname, port):
    engine = create_engine(f'mysql+pymysql://{user_name}:{user_password}@{host_name}:{port}/{dbname}')
    return engine

In [14]:
abbreviation_to_state = {
    'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California',
    'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia',
    'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa',
    'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland',
    'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri',
    'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey',
    'NM': 'New Mexico', 'NY': 'New York', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio',
    'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina',
    'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont',
    'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming'
}
# extraction from csv to be loaded as raw data in staging area
def extract_bank_data():
    eng = engine("localhost", "root", "Layaldbroot1997", "financial_fraud", "3306")
    df = pd.read_csv("bank_fraud_data.csv")
    df.to_sql("f_fraud",con = eng, schema = "financial_fraud", if_exists = 'replace', index=False)
    return

def extract_credit_card():
    eng = engine("localhost", "root", "Layaldbroot1997", "credit_card_fraud", "3306")
    df = pd.read_csv("credit_card_fraud_data.csv")
    df.to_sql("cc_fraud",con = eng, schema = "credit_card_fraud", if_exists = 'replace', index=False)
    return
#extract_bank_data()
#extract_credit_card()

In [15]:
# extraction from staging are to transform
def extract_financial_from_staging():
    conn= connection("localhost", "root", "Layaldbroot1997", "financial_fraud")
    cursor=conn.cursor()
    query="SELECT * FROM financial_fraud.f_fraud"
    cursor.execute(query)
    data=cursor.fetchall()
    column_names=[i[0] for i in cursor.description]
    df=pd.DataFrame(data, columns=column_names)
    return df

def extract_credit_card_from_staging():
    conn= connection("localhost", "root", "Layaldbroot1997", "credit_card_fraud")
    cursor=conn.cursor()
    query="SELECT * FROM credit_card_fraud.cc_fraud"
    cursor.execute(query)
    data=cursor.fetchall()
    column_names=[i[0] for i in cursor.description]
    df=pd.DataFrame(data, columns=column_names)
    return df

In [16]:
# transform data of bank fraud
def transform_financial():
    df = extract_financial_from_staging()
    
    # set fraud type to 1
    df['fraud_type'] = 1
    
    # split timestamp
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    df['Year'] = df['Timestamp'].dt.year
    df['Month'] = df['Timestamp'].dt.month
    df['Day'] = df['Timestamp'].dt.day
    df['Day_of_Week'] = df['Timestamp'].dt.day_name() 
    df['Hour'] = df['Timestamp'].dt.hour
    df['Minute'] = df['Timestamp'].dt.minute
    df['Second'] = df['Timestamp'].dt.second
    
    # split merchant location
    df['m_street'] = df['Location'].str.split('-', expand=True)[0].str.strip()
    df['m_city'] = df['Location'].str.split('-', expand=True)[1].str.split(',', expand=True)[0].str.strip()
    df['m_state'] = df['Location'].str.split(',', expand=True)[1].str.split().str[0].str.strip()
    df['m_zip'] = df['Location'].str.split().str[-1].str.strip()
    df['m_state'] = df['m_state'].replace(abbreviation_to_state)
    
    # split customer address
    df['c_street'] = df['Address'].str.split('-', expand=True)[0].str.strip()
    df['c_city'] = df['Address'].str.split('-', expand=True)[1].str.split(',', expand=True)[0].str.strip()
    df['c_state'] = df['Address'].str.split(',', expand=True)[1].str.split().str[0].str.strip()
    df['c_zip'] = df['Address'].str.split().str[-1].str.strip()
    df['c_state'] = df['c_state'].replace(abbreviation_to_state)
    return df

In [17]:
# initialize the geolocator
geolocator = Nominatim(user_agent="merchant_address")

def get_address_from_lat_lon(latitude, longitude):
    try:
        location = geolocator.reverse((latitude, longitude), exactly_one=True)
        if location:
            address = location.raw['address']
            street = address.get('road', '')
            city = address.get('city', address.get('town', address.get('village', '')))
            state = address.get('state', '')
            zip_code = address.get('postcode', '')
            return street, city, state, zip_code
        else:
            print(f"No location found for {latitude}, {longitude}")
            return None, None, None, None
    except Exception as e:
        print(f"Error retrieving address for {latitude}, {longitude}: {e}")
        return None, None, None, None

def append_address_columns(df):
    # Create new columns for address information
    df['m_street'] = None
    df['m_city'] = None
    df['m_state'] = None
    df['m_zip'] = None

    for index, row in df.iterrows():
        latitude = row['merch_lat']
        longitude = row['merch_long']
        street, city, state, zip_code = get_address_from_lat_lon(latitude, longitude)
        df.at[index, 'm_street'] = street
        df.at[index, 'm_city'] = city
        df.at[index, 'm_state'] = state
        df.at[index, 'm_zip'] = zip_code

    return df

In [18]:
# transform data of credit card fraud
def transform_credit_card():
    df = extract_credit_card_from_staging()
    # remove fraud_ from merchant name
    df['merchant'] = df['merchant'].str.replace('fraud_', '', regex=True).str.replace('-', ' ')
    df = df.drop(columns=['Unnamed'], axis=1)

    # concatenate first and last to have customer name
    df['customer_name'] = df['first'] + ' ' + df['last']
    
    # set fraud type to 2
    df['fraud_type'] = 2

    # split timestamp
    df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'])
    df['Year'] = df['trans_date_trans_time'].dt.year
    df['Month'] = df['trans_date_trans_time'].dt.month
    df['Day'] = df['trans_date_trans_time'].dt.day
    df['Day_of_Week'] = df['trans_date_trans_time'].dt.day_name() 
    df['Hour'] = df['trans_date_trans_time'].dt.hour
    df['Minute'] = df['trans_date_trans_time'].dt.minute
    df['Second'] = df['trans_date_trans_time'].dt.second
    df['state'] = df['state'].replace(abbreviation_to_state)
    
    # generate merchant address details
    df = append_address_columns(df)
    return df


In [19]:
def load_transformed_data():
    eng = engine("localhost", "root", "Layaldbroot1997", "financial_fraud", "3306")
    financial = transform_financial()
    credit_card = transform_credit_card()
    # load transformed data into staging
    financial.to_sql("bank",con = eng, schema = "fraudulent_activities", if_exists = 'replace', index=False)
    credit_card.to_sql("credit_card",con = eng, schema = "fraudulent_activities", if_exists = 'replace', index=False)
    return
load_transformed_data()

Connection successful!
Connection successful!
No location found for 33.384541, -117.954597
No location found for 40.294144, -73.28307099999999
No location found for 40.026776, -72.864038
No location found for 40.211958, -73.30061500000001
No location found for 23.786693, -81.25359
No location found for 42.421976, -125.223482
No location found for 40.308265000000006, -73.510504
No location found for 27.139966, -82.866194
No location found for 39.751135, -72.581032
No location found for 32.843874, -117.746839
No location found for 20.040485, -156.44325700000002
No location found for 40.581546, -72.22094399999999
No location found for 39.948899, -72.88786800000001
No location found for 26.071929, -82.710944
No location found for 30.554686, -81.13521800000001
No location found for 25.753258, -79.78586800000001
No location found for 64.262873, -166.227389
No location found for 37.496804, -75.166995
No location found for 23.806853, -80.809259
No location found for 27.077048, -79.488575
No lo